In [1]:
# Import the required libraries and dependencies

import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
# Set the variables for the Alpaca API and secret keys
load_dotenv()

True

In [3]:
alpaca_api_key = os.environ.get('ALPACA_API_KEY')
alpaca_secret_key = os.environ.get('ALPACA_SECRET_KEY')

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = "v2")

In [4]:
# Set the tickers for Technology - XLK, Health Care - XLV, Financials - XLF,
# Real Estate - XLRE, Energy - XLE, Materials - XLB, Communication - XLC, 
# Consumer Discretionary - XLY, Consumer Staples - XLP, Financials - XLF, 
# Industrials - XLI, Utilities - XLU

tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

# Set timeframe to 1Day
timeframe = '1D'

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# 9+1 to backtest data
start_date = pd.Timestamp("2013-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-05-01", tz="America/New_York").isoformat()

In [5]:
# # Monte Carlo 

# # Import the MCSimulation module from the MCForecastTools library file
# from MCForecastTools import MCSimulation

# # Configure the Monte Carlo simulation to forecast 30 years cumulative returns
# # The weights should be split 40% to AGG and 60% to SPY.
# # Run 500 samples.

# MC_thirtyyear = MCSimulation(
#     portfolio_data = pricing_data,
#     weights = [.40,.60],
#     num_simulation = 500,
#     num_trading_days = 252*30
# )

# # Review the simulation input data
# MC_thirtyyear.portfolio_data.head()

In [6]:
# Use the Alpaca get_bars function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame

portfolio_data = alpaca.get_bars(tickers, timeframe, start = start_date, end = end_date).df
portfolio_data

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2015-12-01 05:00:00+00:00,45.76,46.0600,45.600,46.05,7717036,27667,45.921989,XLB
2015-12-02 05:00:00+00:00,45.91,46.0200,45.280,45.43,8005887,24522,45.556902,XLB
2015-12-03 05:00:00+00:00,45.65,45.7300,44.795,45.03,5828780,28059,45.184572,XLB
2015-12-04 05:00:00+00:00,45.13,45.9400,44.990,45.82,5607746,27545,45.606292,XLB
2015-12-07 05:00:00+00:00,45.57,45.7900,44.710,45.00,6163064,23783,44.993662,XLB
...,...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,146.38,146.6400,144.410,144.43,4247259,50960,145.258770,XLY
2023-04-26 04:00:00+00:00,145.16,145.6300,143.400,143.69,3853004,49809,144.516404,XLY
2023-04-27 04:00:00+00:00,144.70,147.6500,144.600,147.48,4815387,68454,146.631320,XLY


In [7]:
#Check for zero values within the dataset portfolio_data

sum = 0
for i in portfolio_data.iloc[:,3]:
    if i == 0:
        sum = sum + 1 

print(sum)

0


In [8]:
# Reorganize the DataFrame for XLK
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLK_data = portfolio_data[portfolio_data['symbol'] == 'XLK'].drop('symbol', axis=1)
XLK_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,44.14,44.3600,44.045,44.35,7779488,25116,44.230051
2015-12-02 05:00:00+00:00,44.40,44.5650,44.020,44.08,9697088,28153,44.323713
2015-12-03 05:00:00+00:00,44.19,44.3200,43.290,43.50,11854265,33797,43.723955
2015-12-04 05:00:00+00:00,43.63,44.6450,43.540,44.57,12840171,35916,44.328743
2015-12-07 05:00:00+00:00,44.52,44.5577,44.140,44.35,12814944,25604,44.332508
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,146.39,146.6700,143.920,143.97,5531437,70095,145.058985
2023-04-26 04:00:00+00:00,146.50,147.6200,145.760,146.11,6801804,79730,146.450357
2023-04-27 04:00:00+00:00,146.80,149.3150,146.440,149.21,6042384,68503,148.193463


In [9]:
# Reorganize the DataFrame for XLV
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLV_data = portfolio_data[portfolio_data['symbol'] == 'XLV'].drop('symbol', axis=1)
XLV_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,71.48,72.3700,71.3171,72.31,10659529,53601,71.933507
2015-12-02 05:00:00+00:00,72.39,72.6085,71.5900,71.70,9875112,55788,71.985696
2015-12-03 05:00:00+00:00,71.76,71.8500,69.8200,70.13,13191350,65769,70.485965
2015-12-04 05:00:00+00:00,70.44,71.8900,70.2720,71.78,14929777,69939,71.380796
2015-12-07 05:00:00+00:00,71.63,72.0000,71.0100,71.38,8511194,45458,71.346525
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,134.85,135.2600,133.4200,133.61,7752049,71716,134.205579
2023-04-26 04:00:00+00:00,132.43,132.5200,130.9200,131.71,10128825,88575,131.652697
2023-04-27 04:00:00+00:00,131.77,132.5850,130.6200,132.40,10008925,87182,131.662831


In [10]:
# Reorganize the DataFrame for XLV
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLV_data = portfolio_data[portfolio_data['symbol'] == 'XLV'].drop('symbol', axis=1)
XLV_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,71.48,72.3700,71.3171,72.31,10659529,53601,71.933507
2015-12-02 05:00:00+00:00,72.39,72.6085,71.5900,71.70,9875112,55788,71.985696
2015-12-03 05:00:00+00:00,71.76,71.8500,69.8200,70.13,13191350,65769,70.485965
2015-12-04 05:00:00+00:00,70.44,71.8900,70.2720,71.78,14929777,69939,71.380796
2015-12-07 05:00:00+00:00,71.63,72.0000,71.0100,71.38,8511194,45458,71.346525
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,134.85,135.2600,133.4200,133.61,7752049,71716,134.205579
2023-04-26 04:00:00+00:00,132.43,132.5200,130.9200,131.71,10128825,88575,131.652697
2023-04-27 04:00:00+00:00,131.77,132.5850,130.6200,132.40,10008925,87182,131.662831


In [11]:
# Reorganize the DataFrame for XLF
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLF_data = portfolio_data[portfolio_data['symbol'] == 'XLF'].drop('symbol', axis=1)
XLF_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,24.65,24.8400,24.600,24.83,31087166,39743,24.717556
2015-12-02 05:00:00+00:00,24.89,24.8900,24.490,24.54,35618658,47903,24.677192
2015-12-03 05:00:00+00:00,24.68,24.6800,24.060,24.14,48179183,59336,24.285806
2015-12-04 05:00:00+00:00,24.29,24.8200,24.225,24.78,42686845,61492,24.616355
2015-12-07 05:00:00+00:00,24.76,24.7700,24.433,24.55,25492411,40372,24.558467
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,32.93,32.9900,32.560,32.57,58534695,96099,32.724781
2023-04-26 04:00:00+00:00,32.48,32.6600,32.120,32.26,54294597,94784,32.333898
2023-04-27 04:00:00+00:00,32.38,32.8400,32.380,32.78,46398842,90007,32.649997


In [12]:
# Reorganize the DataFrame for XLRE
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLRE_data = portfolio_data[portfolio_data['symbol'] == 'XLRE'].drop('symbol', axis=1)
XLRE_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,30.98,31.2586,30.980,31.2586,5557,35,31.085091
2015-12-02 05:00:00+00:00,31.11,31.2200,30.620,30.6200,4397,27,30.948158
2015-12-03 05:00:00+00:00,30.37,30.3700,30.010,30.0100,776,7,30.155751
2015-12-04 05:00:00+00:00,30.31,30.8300,30.310,30.6200,4378,22,30.703560
2015-12-07 05:00:00+00:00,30.62,30.6200,30.620,30.6200,0,0,0.000000
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,36.87,37.0400,36.720,36.7400,3664353,20378,36.854772
2023-04-26 04:00:00+00:00,36.65,37.0100,36.340,36.4400,4446325,26622,36.651611
2023-04-27 04:00:00+00:00,36.54,37.4050,36.540,37.3200,4278753,26225,37.085132


In [13]:
# Reorganize the DataFrame for XLE
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLE_data = portfolio_data[portfolio_data['symbol'] == 'XLE'].drop('symbol', axis=1)
XLE_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,68.08,68.620,67.9100,68.56,13797038,67795,68.280049
2015-12-02 05:00:00+00:00,68.11,68.370,66.2200,66.44,25428080,132209,67.163013
2015-12-03 05:00:00+00:00,66.76,66.980,64.7900,65.11,24219152,141786,65.678543
2015-12-04 05:00:00+00:00,64.50,65.085,63.4800,64.70,32839278,173452,64.338207
2015-12-07 05:00:00+00:00,63.10,63.190,61.3500,62.24,32800384,165982,62.070845
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,85.44,85.590,84.1600,84.66,15332847,133311,84.651058
2023-04-26 04:00:00+00:00,84.06,84.840,83.0200,83.53,20035092,164481,83.863145
2023-04-27 04:00:00+00:00,83.48,84.040,82.6900,83.83,17057343,143292,83.465898


In [14]:
# Reorganize the DataFrame for XLB
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLB_data = portfolio_data[portfolio_data['symbol'] == 'XLB'].drop('symbol', axis=1)
XLB_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,45.76,46.060,45.600,46.05,7717036,27667,45.921989
2015-12-02 05:00:00+00:00,45.91,46.020,45.280,45.43,8005887,24522,45.556902
2015-12-03 05:00:00+00:00,45.65,45.730,44.795,45.03,5828780,28059,45.184572
2015-12-04 05:00:00+00:00,45.13,45.940,44.990,45.82,5607746,27545,45.606292
2015-12-07 05:00:00+00:00,45.57,45.790,44.710,45.00,6163064,23783,44.993662
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,80.40,80.480,79.450,79.49,5983198,51413,79.796496
2023-04-26 04:00:00+00:00,79.17,79.435,78.285,78.55,5955223,51619,78.830033
2023-04-27 04:00:00+00:00,78.78,79.650,78.325,79.62,5913180,49540,79.024982


In [15]:
# Reorganize the DataFrame for XLC
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLC_data = portfolio_data[portfolio_data['symbol'] == 'XLC'].drop('symbol', axis=1)
XLC_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2018-06-19 04:00:00+00:00,49.70,50.060,49.580,49.96,16588,66,49.816014
2018-06-20 04:00:00+00:00,50.45,50.889,50.450,50.58,189989,912,50.682223
2018-06-21 04:00:00+00:00,50.77,50.850,50.200,50.27,428740,604,50.456106
2018-06-22 04:00:00+00:00,50.59,50.610,50.190,50.49,181638,389,50.467921
2018-06-25 04:00:00+00:00,50.23,50.230,49.015,49.45,2509603,1488,49.214474
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,57.36,57.460,56.720,56.73,5345938,32754,57.024743
2023-04-26 04:00:00+00:00,56.75,57.110,56.065,56.13,6964756,40709,56.481015
2023-04-27 04:00:00+00:00,58.24,59.570,58.100,59.39,9597483,49983,59.132861


In [16]:
# Reorganize the DataFrame for XLY
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLY_data = portfolio_data[portfolio_data['symbol'] == 'XLY'].drop('symbol', axis=1)
XLY_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,81.22,81.6400,80.985,81.61,8187446,44148,81.396035
2015-12-02 05:00:00+00:00,81.73,81.7500,80.740,80.90,6263477,38760,81.183407
2015-12-03 05:00:00+00:00,81.24,81.2400,79.330,79.68,9828903,58541,80.089175
2015-12-04 05:00:00+00:00,79.97,81.3200,79.780,81.22,7032182,44109,80.822906
2015-12-07 05:00:00+00:00,81.10,81.2499,80.320,80.88,6535419,37396,80.740029
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,146.38,146.6400,144.410,144.43,4247259,50960,145.258770
2023-04-26 04:00:00+00:00,145.16,145.6300,143.400,143.69,3853004,49809,144.516404
2023-04-27 04:00:00+00:00,144.70,147.6500,144.600,147.48,4815387,68454,146.631320


In [17]:
# Reorganize the DataFrame for XLP
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLP_data = portfolio_data[portfolio_data['symbol'] == 'XLP'].drop('symbol', axis=1)
XLP_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,49.52,49.8700,49.520,49.84,10807852,23119,49.776266
2015-12-02 05:00:00+00:00,49.77,49.9100,49.520,49.59,7832017,23528,49.718725
2015-12-03 05:00:00+00:00,49.77,49.9500,49.140,49.33,12556022,34239,49.431362
2015-12-04 05:00:00+00:00,49.46,50.4800,49.260,50.44,12783219,37652,50.231542
2015-12-07 05:00:00+00:00,50.63,50.7400,50.380,50.58,9022022,27883,50.548680
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,76.86,77.3700,76.695,76.78,12180191,68021,76.989457
2023-04-26 04:00:00+00:00,76.42,76.7150,76.140,76.18,8779519,53954,76.411361
2023-04-27 04:00:00+00:00,76.22,77.1054,76.170,77.03,11287817,66708,76.748504


In [18]:
# Reorganize the DataFrame for XLF
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLF_data = portfolio_data[portfolio_data['symbol'] == 'XLF'].drop('symbol', axis=1)
XLF_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,24.65,24.8400,24.600,24.83,31087166,39743,24.717556
2015-12-02 05:00:00+00:00,24.89,24.8900,24.490,24.54,35618658,47903,24.677192
2015-12-03 05:00:00+00:00,24.68,24.6800,24.060,24.14,48179183,59336,24.285806
2015-12-04 05:00:00+00:00,24.29,24.8200,24.225,24.78,42686845,61492,24.616355
2015-12-07 05:00:00+00:00,24.76,24.7700,24.433,24.55,25492411,40372,24.558467
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,32.93,32.9900,32.560,32.57,58534695,96099,32.724781
2023-04-26 04:00:00+00:00,32.48,32.6600,32.120,32.26,54294597,94784,32.333898
2023-04-27 04:00:00+00:00,32.38,32.8400,32.380,32.78,46398842,90007,32.649997


In [19]:
# Reorganize the DataFrame for XLI
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLI_data = portfolio_data[portfolio_data['symbol'] == 'XLI'].drop('symbol', axis=1)
XLI_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,54.76,55.175,54.6900,55.00,11475851,38487,54.891929
2015-12-02 05:00:00+00:00,54.91,55.060,54.3250,54.44,13131576,38293,54.598700
2015-12-03 05:00:00+00:00,54.58,54.610,53.5601,53.76,16001355,60179,53.975929
2015-12-04 05:00:00+00:00,53.84,54.570,53.7200,54.56,13891933,53684,54.338047
2015-12-07 05:00:00+00:00,54.39,54.565,54.1000,54.29,9665464,36876,54.281733
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,100.25,100.290,98.9200,98.99,12306937,92127,99.398208
2023-04-26 04:00:00+00:00,98.35,98.610,96.9200,97.12,16058847,116986,97.683737
2023-04-27 04:00:00+00:00,97.41,99.155,97.2600,99.04,14421565,103637,98.254845


In [20]:
# Reorganize the DataFrame for XLU
# Separate ticker data
# tickers = ['XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU']

XLU_data = portfolio_data[portfolio_data['symbol'] == 'XLU'].drop('symbol', axis=1)
XLU_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,42.92,43.250,42.7500,43.16,12551543,44273,43.040413
2015-12-02 05:00:00+00:00,42.99,43.100,42.1900,42.20,12566056,43285,42.440103
2015-12-03 05:00:00+00:00,42.13,42.140,41.7400,41.88,14879220,60857,41.935748
2015-12-04 05:00:00+00:00,42.03,42.530,41.9000,42.47,10961473,43227,42.383786
2015-12-07 05:00:00+00:00,42.37,42.680,42.1701,42.66,13515101,39592,42.524453
...,...,...,...,...,...,...,...
2023-04-25 04:00:00+00:00,69.71,70.280,69.6900,69.90,10251756,75343,69.958057
2023-04-26 04:00:00+00:00,69.36,69.575,68.1800,68.26,14757787,101436,68.600552
2023-04-27 04:00:00+00:00,68.46,69.140,68.4000,69.08,12466677,87745,68.841671


In [21]:
#Check for N/A values within each ETF dataframe
#'XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU'

display(XLK_data.isna().sum())
display(XLV_data.isna().sum())
display(XLF_data.isna().sum())
display(XLRE_data.isna().sum())
display(XLE_data.isna().sum())
display(XLB_data.isna().sum())
display(XLC_data.isna().sum())
display(XLY_data.isna().sum())
display(XLF_data.isna().sum())
display(XLI_data.isna().sum())
display(XLU_data.isna().sum())

open           0
high           0
low            0
close          0
volume         0
trade_count    0
vwap           0
dtype: int64

open           0
high           0
low            0
close          0
volume         0
trade_count    0
vwap           0
dtype: int64

open           0
high           0
low            0
close          0
volume         0
trade_count    0
vwap           0
dtype: int64

open           0
high           0
low            0
close          0
volume         0
trade_count    0
vwap           0
dtype: int64

open           0
high           0
low            0
close          0
volume         0
trade_count    0
vwap           0
dtype: int64

...

open           0
high           0
low            0
close          0
volume         0
trade_count    0
vwap           0
dtype: int64

open           0
high           0
low            0
close          0
volume         0
trade_count    0
vwap           0
dtype: int64

open           0
high           0
low            0
close          0
volume         0
trade_count    0
vwap           0
dtype: int64

open           0
high           0
low            0
close          0
volume         0
trade_count    0
vwap           0
dtype: int64

open           0
high           0
low            0
close          0
volume         0
trade_count    0
vwap           0
dtype: int64

In [22]:
# Break up each Sector ETF dataframe to only include timestamp and close below
#'XLK', 'XLV', 'XLF', 'XLRE', 'XLE', 'XLB', 'XLC', 'XLY', 'XLP', 'XLF', 'XLI', 'XLU'

In [23]:
# For XLK

XLK_close = XLK_data.iloc[:,3]
XLK_close

timestamp
2015-12-01 05:00:00+00:00     44.35
2015-12-02 05:00:00+00:00     44.08
2015-12-03 05:00:00+00:00     43.50
2015-12-04 05:00:00+00:00     44.57
2015-12-07 05:00:00+00:00     44.35
                              ...  
2023-04-25 04:00:00+00:00    143.97
2023-04-26 04:00:00+00:00    146.11
2023-04-27 04:00:00+00:00    149.21
2023-04-28 04:00:00+00:00    150.83
2023-05-01 04:00:00+00:00    150.92
Name: close, Length: 1866, dtype: float64

In [24]:
# For XLV

XLV_close = XLV_data.iloc[:,3]
XLV_close

timestamp
2015-12-01 05:00:00+00:00     72.31
2015-12-02 05:00:00+00:00     71.70
2015-12-03 05:00:00+00:00     70.13
2015-12-04 05:00:00+00:00     71.78
2015-12-07 05:00:00+00:00     71.38
                              ...  
2023-04-25 04:00:00+00:00    133.61
2023-04-26 04:00:00+00:00    131.71
2023-04-27 04:00:00+00:00    132.40
2023-04-28 04:00:00+00:00    133.53
2023-05-01 04:00:00+00:00    134.22
Name: close, Length: 1866, dtype: float64

In [25]:
# For XLF

XLF_close = XLF_data.iloc[:,3]
XLF_close

timestamp
2015-12-01 05:00:00+00:00    24.83
2015-12-02 05:00:00+00:00    24.54
2015-12-03 05:00:00+00:00    24.14
2015-12-04 05:00:00+00:00    24.78
2015-12-07 05:00:00+00:00    24.55
                             ...  
2023-04-25 04:00:00+00:00    32.57
2023-04-26 04:00:00+00:00    32.26
2023-04-27 04:00:00+00:00    32.78
2023-04-28 04:00:00+00:00    33.17
2023-05-01 04:00:00+00:00    33.08
Name: close, Length: 1866, dtype: float64

In [26]:
# For XLRE

XLRE_close = XLRE_data.iloc[:,3]
XLRE_close

timestamp
2015-12-01 05:00:00+00:00    31.2586
2015-12-02 05:00:00+00:00    30.6200
2015-12-03 05:00:00+00:00    30.0100
2015-12-04 05:00:00+00:00    30.6200
2015-12-07 05:00:00+00:00    30.6200
                              ...   
2023-04-25 04:00:00+00:00    36.7400
2023-04-26 04:00:00+00:00    36.4400
2023-04-27 04:00:00+00:00    37.3200
2023-04-28 04:00:00+00:00    37.7500
2023-05-01 04:00:00+00:00    37.4000
Name: close, Length: 1866, dtype: float64

In [27]:
# For XLE

XLE_close = XLE_data.iloc[:,3]
XLE_close

timestamp
2015-12-01 05:00:00+00:00    68.56
2015-12-02 05:00:00+00:00    66.44
2015-12-03 05:00:00+00:00    65.11
2015-12-04 05:00:00+00:00    64.70
2015-12-07 05:00:00+00:00    62.24
                             ...  
2023-04-25 04:00:00+00:00    84.66
2023-04-26 04:00:00+00:00    83.53
2023-04-27 04:00:00+00:00    83.83
2023-04-28 04:00:00+00:00    85.13
2023-05-01 04:00:00+00:00    84.17
Name: close, Length: 1866, dtype: float64

In [28]:
# For XLC

XLC_close = XLC_data.iloc[:,3]
XLC_close

timestamp
2018-06-19 04:00:00+00:00    49.96
2018-06-20 04:00:00+00:00    50.58
2018-06-21 04:00:00+00:00    50.27
2018-06-22 04:00:00+00:00    50.49
2018-06-25 04:00:00+00:00    49.45
                             ...  
2023-04-25 04:00:00+00:00    56.73
2023-04-26 04:00:00+00:00    56.13
2023-04-27 04:00:00+00:00    59.39
2023-04-28 04:00:00+00:00    59.90
2023-05-01 04:00:00+00:00    59.89
Name: close, Length: 1225, dtype: float64

In [29]:
# For XLY

XLY_close = XLY_data.iloc[:,3]
XLY_close

timestamp
2015-12-01 05:00:00+00:00     81.61
2015-12-02 05:00:00+00:00     80.90
2015-12-03 05:00:00+00:00     79.68
2015-12-04 05:00:00+00:00     81.22
2015-12-07 05:00:00+00:00     80.88
                              ...  
2023-04-25 04:00:00+00:00    144.43
2023-04-26 04:00:00+00:00    143.69
2023-04-27 04:00:00+00:00    147.48
2023-04-28 04:00:00+00:00    147.86
2023-05-01 04:00:00+00:00    146.40
Name: close, Length: 1866, dtype: float64

In [30]:
# For XLP

XLP_close = XLP_data.iloc[:,3]
XLP_close

timestamp
2015-12-01 05:00:00+00:00    49.84
2015-12-02 05:00:00+00:00    49.59
2015-12-03 05:00:00+00:00    49.33
2015-12-04 05:00:00+00:00    50.44
2015-12-07 05:00:00+00:00    50.58
                             ...  
2023-04-25 04:00:00+00:00    76.78
2023-04-26 04:00:00+00:00    76.18
2023-04-27 04:00:00+00:00    77.03
2023-04-28 04:00:00+00:00    77.44
2023-05-01 04:00:00+00:00    77.50
Name: close, Length: 1866, dtype: float64

In [31]:
# For XLF

XLF_close = XLF_data.iloc[:,3]
XLF_close

timestamp
2015-12-01 05:00:00+00:00    24.83
2015-12-02 05:00:00+00:00    24.54
2015-12-03 05:00:00+00:00    24.14
2015-12-04 05:00:00+00:00    24.78
2015-12-07 05:00:00+00:00    24.55
                             ...  
2023-04-25 04:00:00+00:00    32.57
2023-04-26 04:00:00+00:00    32.26
2023-04-27 04:00:00+00:00    32.78
2023-04-28 04:00:00+00:00    33.17
2023-05-01 04:00:00+00:00    33.08
Name: close, Length: 1866, dtype: float64

In [32]:
# For XLI

XLI_close = XLI_data.iloc[:,3]
XLI_close

timestamp
2015-12-01 05:00:00+00:00     55.00
2015-12-02 05:00:00+00:00     54.44
2015-12-03 05:00:00+00:00     53.76
2015-12-04 05:00:00+00:00     54.56
2015-12-07 05:00:00+00:00     54.29
                              ...  
2023-04-25 04:00:00+00:00     98.99
2023-04-26 04:00:00+00:00     97.12
2023-04-27 04:00:00+00:00     99.04
2023-04-28 04:00:00+00:00    100.00
2023-05-01 04:00:00+00:00    100.54
Name: close, Length: 1866, dtype: float64

In [33]:
# For XLU

XLU_close = XLU_data.iloc[:,3]
XLU_close

timestamp
2015-12-01 05:00:00+00:00    43.16
2015-12-02 05:00:00+00:00    42.20
2015-12-03 05:00:00+00:00    41.88
2015-12-04 05:00:00+00:00    42.47
2015-12-07 05:00:00+00:00    42.66
                             ...  
2023-04-25 04:00:00+00:00    69.90
2023-04-26 04:00:00+00:00    68.26
2023-04-27 04:00:00+00:00    69.08
2023-04-28 04:00:00+00:00    68.98
2023-05-01 04:00:00+00:00    69.10
Name: close, Length: 1866, dtype: float64